In [1]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [736 kB]
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,569 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,079 kB]
Get:13 http://

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
# Initialize findspark
import findspark
findspark.init()

RDD examples

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("Spark RDD example").getOrCreate()
data=[("Z", 1),("A", 20),("B", 30),("C", 40),("B", 30),("B", 60)]
inputRDD = spark.sparkContext.parallelize(data)
inputRDD.toDF().show()

+---+---+
| _1| _2|
+---+---+
|  Z|  1|
|  A| 20|
|  B| 30|
|  C| 40|
|  B| 30|
|  B| 60|
+---+---+



In [11]:
listRdd = spark.sparkContext.parallelize([1,2,3,4,5,3,2])

In [12]:
#aggregate 1
seqOp = (lambda x, y: x + y)
combOp = (lambda x, y: x + y)
agg=listRdd.aggregate(0, seqOp, combOp)
print(agg)

20


In [13]:
#aggregate 2
seqOp2 = (lambda x, y: (x[0] + y, x[1] + 1))
combOp2 = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
agg2=listRdd.aggregate((0, 0), seqOp2, combOp2)
print(agg2)

(20, 7)


In [14]:
agg2=listRdd.treeAggregate(0,seqOp, combOp)
print(agg2)

20


In [15]:
#fold
from operator import add
foldRes=listRdd.fold(0, add)
print(foldRes)

20


In [16]:
#reduce
redRes=listRdd.reduce(add)
print(redRes)

20


In [17]:
#treeReduce. This is similar to reduce
add = lambda x, y: x + y
redRes=listRdd.treeReduce(add)
print(redRes)

20


In [18]:
#Collect
data = listRdd.collect()
print(data)

[1, 2, 3, 4, 5, 3, 2]


In [19]:
#count, countApprox, countApproxDistinct
print("Count : "+str(listRdd.count()))
print("countApprox : "+str(listRdd.countApprox(1200)))
print("countApproxDistinct : "+str(listRdd.countApproxDistinct()))
print("countApproxDistinct : "+str(inputRDD.countApproxDistinct()))

Count : 7
countApprox : 7
countApproxDistinct : 5
countApproxDistinct : 5


In [20]:
#countByValue, countByValueApprox
print("countByValue :  "+str(listRdd.countByValue()))

countByValue :  defaultdict(<class 'int'>, {1: 1, 2: 2, 3: 2, 4: 1, 5: 1})


In [21]:
#first
print("first :  "+str(listRdd.first()))
print("first :  "+str(inputRDD.first()))

first :  1
first :  ('Z', 1)


In [22]:
#top
print("top : "+str(listRdd.top(2)))
print("top : "+str(inputRDD.top(2)))

top : [5, 4]
top : [('Z', 1), ('C', 40)]


In [23]:
#min
print("min :  "+str(listRdd.min()))
print("min :  "+str(inputRDD.min()))

min :  1
min :  ('A', 20)


In [24]:
#max
print("max :  "+str(listRdd.max()))
print("max :  "+str(inputRDD.max()))

max :  5
max :  ('Z', 1)


In [26]:
#take, takeOrdered, takeSample
print("take : "+str(listRdd.take(2)))
print("takeOrdered : "+ str(listRdd.takeOrdered(2)))
print("take : "+str(listRdd.takeSample(False, 2)))

take : [1, 2]
takeOrdered : [1, 2]
take : [3, 2]
